In [1]:
import requests
import tqdm

api_key = '3ca9004a926faf0344572c21df9df1107c2f626b'

In [2]:
# 고유번호

import zipfile
from io import BytesIO

url_unique = "https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={}"
resq = requests.get(url_unique.format(api_key))
z = zipfile.ZipFile(BytesIO(resq.content))

In [3]:
z.namelist()

['CORPCODE.xml']

In [4]:
import xmltodict
import json

data_xml = z.read('CORPCODE.xml').decode('utf-8')
data_odict = xmltodict.parse(data_xml)
data_dict = json.loads(json.dumps(data_odict))
data = data_dict.get('result', {}).get('list')

In [5]:
# 상장사만 #code_dict에 저장되어 있음
code_list = []
for d in data:
    if d['stock_code'] != None:
        code_list.append([d['corp_code'], d['corp_name'],d['stock_code']])

import pandas as pd

df_corp = pd.DataFrame(data=code_list)

In [6]:
df_corp.columns = ['corp_code','corp_name','stock_code']

df_corp

,corp_code,corp_name,stock_code
0,00260985,한빛네트,036720
1,00264529,엔플렉스,040130
2,00358545,동서정보기술,055000
3,00231567,애드모바일,032600
4,00247939,씨모스,037600
...,...,...,...
3528,00185505,제일바이오,052670
3529,00993931,동양파일,228340
3530,00115287,동방,004140
3531,00557933,진도,088790


In [7]:
df_corp['stock_code'] = "A" + df_corp['stock_code'].apply(lambda x: x.zfill(6))
df_corp.columns = ["Dart코드","회사명","기업공시코드"]
df_corp = df_corp.sort_values("기업공시코드").reset_index(drop=True)

In [8]:
df_corp

,Dart코드,회사명,기업공시코드
0,00149293,신한은행,A000010
1,00119195,동화약품,A000020
2,00254045,우리은행,A000030
3,00112378,KR모터스,A000040
4,00101628,경방,A000050
...,...,...,...
3528,01338724,에스앤케이,A950180
3529,01416235,미투젠,A950190
3530,01442115,소마젠,A950200
3531,01510489,프레스티지바이오파마,A950210


In [ ]:
df_corp.to_csv("기업명_기업코드.csv")

In [9]:
# 날짜 인덱스 생성하기

# 시작 인덱스
date_list = pd.date_range('20150101', '20220101', freq='3MS').astype('str').tolist()

new_date_list = []
for d in date_list:
    new_date_list.append(d.replace('-',''))


# 종료 인덱스
date_list2= pd.date_range('20150301', '20220101', freq='3M').astype('str').tolist()

new_date_list2 = []
for d in date_list2:
    new_date_list2.append(d.replace('-',''))

new_date_list2.append('20220331')

final_date_list = list(zip(new_date_list, new_date_list2))

In [12]:
# 공시검색 개발가이드 (공시정보)
url = "https://opendart.fss.or.kr/api/list.json"
l_data = []

for start, end in tqdm.tqdm(final_date_list):

    param_dict = {'crtfc_key': api_key,
              'bgn_de' : start, 'end_de': end,
              'pblntf_ty': 'B',
              'pblntf_detail_ty':'B001', 'corp_cls':'K',
              'page_count':'100'}

    req = requests.get(url, params=param_dict)
    data_json = req.json()
    # print(data_json)

    for dict in data_json['list']:
       l_data.append([dict['corp_code'], dict['corp_name'], dict['stock_code'], dict['rcept_dt'], dict['report_nm']])

l_data

100%|██████████| 29/29 [00:47<00:00,  1.64s/it]


[['00260611', '크로바하이텍', '043590', '20150309', '[첨부추가]주요사항보고서(자기주식취득신탁계약체결결정)'],
 ['00121686', '엠케이전자', '033160', '20150127', '[첨부추가]주요사항보고서(자기주식취득신탁계약체결결정)'],
 ['00136642', '엠벤처투자', '019590', '20150331', '[기재정정]주요사항보고서(전환사채권발행결정)'],
 ['01046708', '바디텍메드', '206640', '20150331', '주요사항보고서(합병결정)'],
 ['00370200', '와이오엠', '066430', '20150331', '[기재정정]주요사항보고서(중요한자산양수도결정)'],
 ['00136642', '엠벤처투자', '019590', '20150331', '주요사항보고서(전환사채권발행결정)'],
 ['00407036', '선바이오', '067370', '20150331', '주요사항보고서(합병결정)'],
 ['00367695', '대한그린파워', '060900', '20150331', '[기재정정]주요사항보고서(유상증자결정)'],
 ['00367695', '대한그린파워', '060900', '20150330', '주요사항보고서(유상증자결정)'],
 ['00378363', '3S', '060310', '20150330', '주요사항보고서(전환사채권발행결정)'],
 ['00216762', '한양이엔지', '045100', '20150330', '[기재정정]주요사항보고서(자기주식취득신탁계약체결결정)'],
 ['00268251', '다산네트웍스', '039560', '20150330', '[기재정정]주요사항보고서(분할(분할합병)결정)'],
 ['00476498', '컴투스', '078340', '20150330', '주요사항보고서(자기주식처분결정)'],
 ['00216762', '한양이엔지', '045100', '20150330', '[기재정정]주요사항보고서(자기주식취득신탁계약체결결정)']

In [27]:
stock_code_list = df_corp["corp_code"].values.tolist()

In [20]:
from tqdm.contrib import tzip
import time

In [26]:
param = {"crtfc_key":api_key,
         "corp_code":"115450",
         "bgn_de":"20220101",
         "end_de":"20230107"}

req = requests.get(url, params=param)
data_json = req.json()

print(data_json)

{'status': '013', 'message': '조회된 데이타가 없습니다.'}


In [30]:
url = "https://opendart.fss.or.kr/api/piicDecsn.json"

l = []

for code, count in tzip(stock_code_list[700:], range(len(stock_code_list[700:]))):
    param = {"crtfc_key":api_key,
             "corp_code":code,
             "bgn_de":"20220101",
             "end_de":"20221231"}
    
    req = requests.get(url, params=param)
    data_json = req.json()

    if data_json['status'] == '000':
        l.append(data_json["list"])
        print(data_json)
    if int(count) == 20:
        time.sleep(1)

  0%|          | 0/2833 [00:00<?, ?it/s]

{'status': '000', 'message': '정상', 'list': [{'rcept_no': '20220812000620', 'corp_cls': 'K', 'corp_code': '01245062', 'corp_name': '코오롱티슈진', 'nstk_ostk_cnt': '794,000', 'nstk_estk_cnt': '-', 'fv_ps': '0', 'bfic_tisstk_ostk': '12,984,010', 'bfic_tisstk_estk': '117,647', 'fdpp_fclt': '-', 'fdpp_bsninh': '-', 'fdpp_op': '38,798,810,000', 'fdpp_dtrp': '-', 'fdpp_ocsa': '-', 'fdpp_etc': '-', 'ic_mthn': '제3자배정증자', 'ssl_at': 'N', 'ssl_bgd': '-', 'ssl_edd': '-'}]}
{'status': '000', 'message': '정상', 'list': [{'rcept_no': '20220727000275', 'corp_cls': 'K', 'corp_code': '01107665', 'corp_name': '크리스탈신소재', 'nstk_ostk_cnt': '24,060,413', 'nstk_estk_cnt': '-', 'fv_ps': '613', 'bfic_tisstk_ostk': '71,830,626', 'bfic_tisstk_estk': '-', 'fdpp_fclt': '-', 'fdpp_bsninh': '-', 'fdpp_op': '-', 'fdpp_dtrp': '-', 'fdpp_ocsa': '34,887,600,000', 'fdpp_etc': '-', 'ic_mthn': '제3자배정증자', 'ssl_at': 'N', 'ssl_bgd': '-', 'ssl_edd': '-'}]}
{'status': '000', 'message': '정상', 'list': [{'rcept_no': '20221031000296', 'corp

KeyboardInterrupt: 